## 데이터셋 병합
- 병합 데이터셋
  - `data\extracted\MergedRoute.csv`
  - `data\extracted\modified_route.csv`
  - `data\extracted\merged_route_check_04_03.csv`
- 최종 데이터 저장 경로
  - `data\extracted\merged_route_final.csv`

In [1]:
import sys
import os

from os import listdir
from os.path import isfile, join
from pathlib import Path

root = Path(os.getcwd()).parent.parent
sys.path.append(str(root))

In [2]:
import pandas as pd
import numpy as np
import re

### `MergedRoute.csv` 전처리 더 필요한 부분

In [3]:
# merged_df 확인
merged_path = join(root, 'data', 'extracted', 'MergedRoute.csv')
merged_df = pd.read_csv(merged_path)
merged_df.head()

,patient_id,city,infection_case,date,type,latitude,longitude,row,col
0,1000000001,jung,overseas inflow,2020-01-24,hospital,37.567241,127.005659,147,152
1,1000000002,seongbuk,overseas inflow,2020-01-25,etc,37.592560,127.017048,119,161
2,1000000002,seongbuk,overseas inflow,2020-01-26,store,37.591810,127.016822,120,161
3,1000000002,seongdong,overseas inflow,2020-01-26,public_transportation,37.563992,127.029534,151,170
4,1000000002,seongbuk,overseas inflow,2020-01-26,public_transportation,37.590330,127.015221,121,160


In [4]:
# 중복되는 type 줄이기 위한 type.csv
type_path = join(root, 'data', 'extracted', 'type.csv')
normal_type_df = pd.read_csv(type_path)

normal_type_df.head()

,type,to_replace
0,hospital,NaN
1,etc,NaN
2,store,NaN
3,public_transportation,NaN
4,restaurant,NaN


In [5]:
# 중복된 type column value 변경
indices_to_del = []

for index, row in merged_df.iterrows():
    location_type = row['type']
    
    normal_row = normal_type_df[normal_type_df['type'] == location_type]
    if normal_row.empty:
        indices_to_del.append(index)
        continue
    else:
        normal_row = normal_row.iloc[0]
        
    if not pd.isnull(normal_row['to_replace']):
        merged_df.loc[index, 'type'] = normal_row['to_replace']
        
for index in indices_to_del:
    merged_df = modified_route.drop(index=index)
    
merged_df = merged_df.reset_index(drop=True)
merged_df.head()

,patient_id,city,infection_case,date,type,latitude,longitude,row,col
0,1000000001,jung,overseas inflow,2020-01-24,hospital,37.567241,127.005659,147,152
1,1000000002,seongbuk,overseas inflow,2020-01-25,etc,37.592560,127.017048,119,161
2,1000000002,seongbuk,overseas inflow,2020-01-26,store,37.591810,127.016822,120,161
3,1000000002,seongdong,overseas inflow,2020-01-26,public_transportation,37.563992,127.029534,151,170
4,1000000002,seongbuk,overseas inflow,2020-01-26,public_transportation,37.590330,127.015221,121,160


### `modified_route.csv` 전처리 더 필요한 부분

In [6]:
# modified_df 확인
modified_path = join(root, 'data', 'extracted', 'modified_route.csv')
modified_df = pd.read_csv(modified_path)
modified_df.head()

,date,patNo,reason,route,type,latitude,longitude
0,2020-04-17,10779,해외 접촉,미국 LA 출발 아시아나항공(OZ201) 이용해 인천공항에 입국,airport,33.941589,-118.408530
1,2020-05-01,10789,해외 접촉,"인천국제공항(대한항공 KE8956, 좌석 55C)",airport,37.460191,126.440696
2,2020-05-01,10789,해외 접촉,공항,airport,37.442055,127.108156
3,2020-04-28,10790,해외 접촉,인천공항 입국(미국),airport,37.460191,126.440696
4,2020-05-02,10839,이태원 클럽 관련,"킹클럽(우사단로, 클럽), 퀸(우사단로, 주점)",bar,37.533634,126.995761


In [7]:
# city 구하기 위해 status file 확인
status_path = join(root, 'data', 'extracted', 'corona_status.csv')
status_df = pd.read_csv(status_path)
status_df.head()

,거주지,여행력,접촉력,퇴원현황,확진일,환자 번호
0,타시도,-,확인 중,NaN,9.20.,23042
1,서초,-,강남구 소재 대우디오빌,NaN,9.20.,23024
2,기타,-,해외 접촉 추정,NaN,9.20.,23016
3,타시도,-,관악구 소재 사우나,NaN,9.20.,23027
4,관악구,-,기타 확진자 접촉,NaN,9.20.,23026


In [8]:
# modified_df의 patNo column과 status_df의 환자 번호 column 비교해서 거주지 복사
indices_to_del = []

for index, row in modified_df.iterrows():
    patient_id = row['patNo']
    
    patient_in_status = status_df[status_df['환자 번호'] == patient_id]
    if patient_in_status.empty:
        indices_to_del.append(index)
        continue
    
    modified_df.loc[index, 'city'] = patient_in_status.iloc[0]['거주지']
    
for index in indices_to_del:
    modified_df = modified_df.drop(index=index)
    
modified_df = modified_df.reset_index(drop=True)
modified_df.head()

,date,patNo,reason,route,type,latitude,longitude,city
0,2020-04-17,10779,해외 접촉,미국 LA 출발 아시아나항공(OZ201) 이용해 인천공항에 입국,airport,33.941589,-118.408530,성동구
1,2020-05-01,10789,해외 접촉,"인천국제공항(대한항공 KE8956, 좌석 55C)",airport,37.460191,126.440696,강동구
2,2020-05-01,10789,해외 접촉,공항,airport,37.442055,127.108156,강동구
3,2020-04-28,10790,해외 접촉,인천공항 입국(미국),airport,37.460191,126.440696,용산구
4,2020-05-02,10839,이태원 클럽 관련,"킹클럽(우사단로, 클럽), 퀸(우사단로, 주점)",bar,37.533634,126.995761,용산구


In [9]:
# 한국어-영어 구
names_path = join(root, 'data', 'raw', 'names.csv')
names_df = pd.read_csv(names_path)

names_df.head()

,korean,english
0,강남,gangnam
1,강동,gangdong
2,강북,gangbuk
3,강서,gangseo
4,관악,gwanak


In [10]:
# 한국어로 된 city 영어로 바꾸기
for index, row in modified_df.iterrows():
    city = row['city']
    if city[-1] == '구': city = city[0:len(city) - 1]
    
    name_row = names_df[names_df['korean'] == city]
    if name_row.empty:
        new_city = 'other'
    else:
        new_city = name_row.iloc[0]['english']
        
    modified_df.loc[index, 'city'] = new_city
    
modified_df.head()

,date,patNo,reason,route,type,latitude,longitude,city
0,2020-04-17,10779,해외 접촉,미국 LA 출발 아시아나항공(OZ201) 이용해 인천공항에 입국,airport,33.941589,-118.408530,seongdong
1,2020-05-01,10789,해외 접촉,"인천국제공항(대한항공 KE8956, 좌석 55C)",airport,37.460191,126.440696,gangdong
2,2020-05-01,10789,해외 접촉,공항,airport,37.442055,127.108156,gangdong
3,2020-04-28,10790,해외 접촉,인천공항 입국(미국),airport,37.460191,126.440696,yongsan
4,2020-05-02,10839,이태원 클럽 관련,"킹클럽(우사단로, 클럽), 퀸(우사단로, 주점)",bar,37.533634,126.995761,yongsan


In [11]:
# 필요한 column만 따로 빼내고, 이름 규격대로 변경
columns = ['date', 'patNo', 'reason', 'type', 'latitude', 'longitude', 'city']
modified_df = modified_df[columns]

new_columns = {'patNo': 'patient_id', 'reason': 'infection_case'}
modified_df.rename(columns=new_columns, inplace=True)
modified_df.head()

,date,patient_id,infection_case,type,latitude,longitude,city
0,2020-04-17,10779,해외 접촉,airport,33.941589,-118.408530,seongdong
1,2020-05-01,10789,해외 접촉,airport,37.460191,126.440696,gangdong
2,2020-05-01,10789,해외 접촉,airport,37.442055,127.108156,gangdong
3,2020-04-28,10790,해외 접촉,airport,37.460191,126.440696,yongsan
4,2020-05-02,10839,이태원 클럽 관련,bar,37.533634,126.995761,yongsan


In [12]:
# 중복되는 type 줄이기 위한 type.csv
type_path = join(root, 'data', 'extracted', 'type.csv')
normal_type_df = pd.read_csv(type_path)

normal_type_df.head()

,type,to_replace
0,hospital,NaN
1,etc,NaN
2,store,NaN
3,public_transportation,NaN
4,restaurant,NaN


In [13]:
# 중복된 type column value 변경
indices_to_del = []

for index, row in modified_df.iterrows():
    location_type = row['type']
    
    normal_row = normal_type_df[normal_type_df['type'] == location_type]
    if normal_row.empty:
        indices_to_del.append(index)
        continue
    else:
        normal_row = normal_row.iloc[0]
        
    if not pd.isnull(normal_row['to_replace']):
        modified_df.loc[index, 'type'] = normal_row['to_replace']
        
for index in indices_to_del:
    modified_df = modified_route.drop(index=index)
    
modified_df = modified_df.reset_index(drop=True)
modified_df.head()

,date,patient_id,infection_case,type,latitude,longitude,city
0,2020-04-17,10779,해외 접촉,airport,33.941589,-118.408530,seongdong
1,2020-05-01,10789,해외 접촉,airport,37.460191,126.440696,gangdong
2,2020-05-01,10789,해외 접촉,airport,37.442055,127.108156,gangdong
3,2020-04-28,10790,해외 접촉,airport,37.460191,126.440696,yongsan
4,2020-05-02,10839,이태원 클럽 관련,bar,37.533634,126.995761,yongsan


### `merged_route_check_04_03.csv` 전처리 더 필요한 부분

In [14]:
# last_df 확인
last_path = join(root, 'data', 'extracted', 'merged_route_check_04_03.csv')
last_df = pd.read_csv(last_path)
last_df.head()

,date,reason,from,location_address,location_type,patient_id,region,lat,lng,type
0,2020-08-22,강남구 역삼동 모임,dongdaemoon,NaN,의료기관,107,휘경1동,37.587895,127.065322,health
1,2020-08-22,강남구 역삼동 모임,dongdaemoon,NaN,약국,107,휘경1동,37.590728,127.062461,health
2,2020-08-22,강남구 역삼동 모임,dongdaemoon,NaN,마트,107,휘경1동,37.594651,127.066109,food
3,2020-08-22,강남구 사무실 관련,dongdaemoon,NaN,생활용품판매점,104,용신동,37.577396,127.031297,home_goods_store
4,2020-08-22,강남구 사무실 관련,dongdaemoon,NaN,이동통신기기판매점,103,장안2동,37.586383,127.043078,point_of_interest


In [15]:
# 한글 구 영어로 바꾸기 위한 names
names_path = join(root, 'data', 'raw', 'names.csv')
names_df = pd.read_csv(names_path)
names_df.head()

,korean,english
0,강남,gangnam
1,강동,gangdong
2,강북,gangbuk
3,강서,gangseo
4,관악,gwanak


In [16]:
# from, patient_id 내용 확인하고 city column 추가
for index, row in last_df.iterrows():
    region = row['from']
    patient_id = row['patient_id']
    
    # patient_id가 숫자로만 이루어진 경우
    if re.search(r'^\d+$', patient_id) is not None:
        last_df.loc[index, 'city'] = region
    else:
        kor_region = re.search(r'[가-힣]+(?=\d+)', patient_id).group()
        name_row = names_df[names_df['korean'] == kor_region].iloc[0]
        eng_region = name_row['english']
        last_df.loc[index, 'city'] = eng_region
        
last_df.head()

,date,reason,from,location_address,location_type,patient_id,region,lat,lng,type,city
0,2020-08-22,강남구 역삼동 모임,dongdaemoon,NaN,의료기관,107,휘경1동,37.587895,127.065322,health,dongdaemoon
1,2020-08-22,강남구 역삼동 모임,dongdaemoon,NaN,약국,107,휘경1동,37.590728,127.062461,health,dongdaemoon
2,2020-08-22,강남구 역삼동 모임,dongdaemoon,NaN,마트,107,휘경1동,37.594651,127.066109,food,dongdaemoon
3,2020-08-22,강남구 사무실 관련,dongdaemoon,NaN,생활용품판매점,104,용신동,37.577396,127.031297,home_goods_store,dongdaemoon
4,2020-08-22,강남구 사무실 관련,dongdaemoon,NaN,이동통신기기판매점,103,장안2동,37.586383,127.043078,point_of_interest,dongdaemoon


In [17]:
# 한국어-영어 location_type 변경 위한 kor_type_df 확인
kor_type_path = join(root, 'data', 'extracted', 'kor_type.csv')
kor_type_df = pd.read_csv(kor_type_path)
kor_type_df.head()

,type,eng_type
0,의료기관,hospital
1,약국,pharmacy
2,마트,supermarket
3,생활용품판매점,store
4,이동통신기기판매점,store


In [18]:
# last_df에서 한국어로 된 location_type 영어로 변경
for index, row in last_df.iterrows():
    type_row = kor_type_df.loc[kor_type_df['type'] == row['location_type']]
    last_df.loc[index, 'type'] = type_row.iloc[0]['eng_type']
    
last_df.head()

,date,reason,from,location_address,location_type,patient_id,region,lat,lng,type,city
0,2020-08-22,강남구 역삼동 모임,dongdaemoon,NaN,의료기관,107,휘경1동,37.587895,127.065322,hospital,dongdaemoon
1,2020-08-22,강남구 역삼동 모임,dongdaemoon,NaN,약국,107,휘경1동,37.590728,127.062461,pharmacy,dongdaemoon
2,2020-08-22,강남구 역삼동 모임,dongdaemoon,NaN,마트,107,휘경1동,37.594651,127.066109,supermarket,dongdaemoon
3,2020-08-22,강남구 사무실 관련,dongdaemoon,NaN,생활용품판매점,104,용신동,37.577396,127.031297,store,dongdaemoon
4,2020-08-22,강남구 사무실 관련,dongdaemoon,NaN,이동통신기기판매점,103,장안2동,37.586383,127.043078,store,dongdaemoon


In [19]:
# 필요한 column만 따로 빼내고, 이름 규격대로 변경
columns = ['date', 'reason', 'patient_id', 'lat', 'lng', 'type', 'city']
last_df = last_df[columns]

new_columns = {'reason': 'infection_case', 'lat': 'latitude', 'lng': 'longitude'}
last_df.rename(columns=new_columns, inplace=True)
last_df.head()

,date,infection_case,patient_id,latitude,longitude,type,city
0,2020-08-22,강남구 역삼동 모임,107,37.587895,127.065322,hospital,dongdaemoon
1,2020-08-22,강남구 역삼동 모임,107,37.590728,127.062461,pharmacy,dongdaemoon
2,2020-08-22,강남구 역삼동 모임,107,37.594651,127.066109,supermarket,dongdaemoon
3,2020-08-22,강남구 사무실 관련,104,37.577396,127.031297,store,dongdaemoon
4,2020-08-22,강남구 사무실 관련,103,37.586383,127.043078,store,dongdaemoon


### merged_df, modified_df, last_df 병합

In [20]:
final_df = pd.concat([merged_df, modified_df, last_df], axis=0, ignore_index=True)
final_df = final_df.reset_index(drop=True)
final_df.head()

,patient_id,city,infection_case,date,type,latitude,longitude,row,col
0,1000000001,jung,overseas inflow,2020-01-24,hospital,37.567241,127.005659,147.0,152.0
1,1000000002,seongbuk,overseas inflow,2020-01-25,etc,37.592560,127.017048,119.0,161.0
2,1000000002,seongbuk,overseas inflow,2020-01-26,store,37.591810,127.016822,120.0,161.0
3,1000000002,seongdong,overseas inflow,2020-01-26,public_transportation,37.563992,127.029534,151.0,170.0
4,1000000002,seongbuk,overseas inflow,2020-01-26,public_transportation,37.590330,127.015221,121.0,160.0


In [21]:
# row, col 없는 경우 값 채워놓기
def to_grid(lat, lon):
    row = 256
    column = 256
    seoul_start_lat = 37.698098
    seoul_start_lon = 126.799791
    seoul_end_lat = 37.472494
    seoul_end_lon = 127.142928
    lat_distance = seoul_end_lat - seoul_start_lat 
    lon_distance = seoul_end_lon - seoul_start_lon

    if seoul_end_lat > lat or lat > seoul_start_lat:
        return -1, -1
    if seoul_start_lon > lon or lon > seoul_end_lon:
        return -1, -1

    lat_distance2= lat - seoul_start_lat
    lon_distance2 = lon - seoul_start_lon
    row = (lat_distance2 * 255) / lat_distance
    col = (lon_distance2 * 255) / lon_distance

    return int(row), int(col)

for index, row in final_df.iterrows():
    if not pd.isnull(row['row']): continue
        
    latitude = row['latitude']
    longitude = row['longitude']
    row_value, col_value = to_grid(latitude, longitude)
    final_df.loc[index, 'row'] = row_value
    final_df.loc[index, 'col'] = col_value
    
final_df.head()

,patient_id,city,infection_case,date,type,latitude,longitude,row,col
0,1000000001,jung,overseas inflow,2020-01-24,hospital,37.567241,127.005659,147.0,152.0
1,1000000002,seongbuk,overseas inflow,2020-01-25,etc,37.592560,127.017048,119.0,161.0
2,1000000002,seongbuk,overseas inflow,2020-01-26,store,37.591810,127.016822,120.0,161.0
3,1000000002,seongdong,overseas inflow,2020-01-26,public_transportation,37.563992,127.029534,151.0,170.0
4,1000000002,seongbuk,overseas inflow,2020-01-26,public_transportation,37.590330,127.015221,121.0,160.0


In [22]:
# infection_case column values 정규화하기 위한 파일
infection_path = join(root, 'data', 'extracted', 'infection_case.csv')
infection_df = pd.read_csv(infection_path)
infection_df.head()

,infection_case,to_replace
0,overseas inflow,NaN
1,contact with patient,NaN
2,Seongdong-gu APT,NaN
3,etc,NaN
4,Eunpyeong St. Mary's Hospital,NaN


In [23]:
# infection_case 정리
count = 0
indices_to_del = []

for index, row in final_df.iterrows():
    infection_case = row['infection_case']
    if pd.isnull(infection_case):
        indices_to_del.append(index)
        continue
    
    infection_row = infection_df.loc[infection_df['infection_case'] == infection_case].iloc[0]
    
    if pd.isnull(infection_row['to_replace']): continue
    final_df.loc[index, 'infection_case'] = infection_row['to_replace']
    count += 1
    
for index in indices_to_del:
    final_df = final_df.drop(index=index)
    
final_df = final_df.reset_index(drop=True)

print('%d rows are chaned' % count)
final_df.head()

4747 rows are chaned


,patient_id,city,infection_case,date,type,latitude,longitude,row,col
0,1000000001,jung,overseas inflow,2020-01-24,hospital,37.567241,127.005659,147.0,152.0
1,1000000002,seongbuk,overseas inflow,2020-01-25,etc,37.592560,127.017048,119.0,161.0
2,1000000002,seongbuk,overseas inflow,2020-01-26,store,37.591810,127.016822,120.0,161.0
3,1000000002,seongdong,overseas inflow,2020-01-26,public_transportation,37.563992,127.029534,151.0,170.0
4,1000000002,seongbuk,overseas inflow,2020-01-26,public_transportation,37.590330,127.015221,121.0,160.0


In [24]:
# row, col 범위 잘못된 부분 삭제
indices_to_del = []

for index, route in final_df.iterrows():
    row = route['row']
    col = route['col']
    
    if pd.isnull(row) or pd.isnull(col):
        print('null')
        indices_to_del.append(index)
        continue
    if row >= 256 or col >= 256:
        indices_to_del.append(index)
        continue
    if row < 0 or col < 0:
        indices_to_del.append(index)
        continue

for index in indices_to_del:
    final_df = final_df.drop(index=index)
    
print('len after delete: %d' % len(final_df.index))

len after delete: 6924


In [25]:
# date column 정규화
for index, row in final_df.iterrows():
    date_str = row['date']
    new_date = re.search(r'\d+-\d+-\d+', date_str).group()
    if date_str == new_date: continue
    final_df.loc[index, 'date'] = new_date

In [26]:
# 날짜 순으로 데이터 정렬
final_df = final_df.sort_values(by='date')
final_df = final_df.reset_index(drop=True)
final_df

,patient_id,city,infection_case,date,type,latitude,longitude,row,col
0,2000000001,gangnam,overseas inflow,2020-01-22,hospital,37.524355,127.027948,196.0,169.0
1,1400000003,yongsan,etc,2020-01-23,etc,37.536606,126.977140,182.0,131.0
2,2000000001,gangnam,overseas inflow,2020-01-23,store,37.527752,127.019480,192.0,163.0
3,1000000001,jung,overseas inflow,2020-01-24,hospital,37.567241,127.005659,147.0,152.0
4,1400000003,jongno,etc,2020-01-24,etc,37.579617,126.977041,133.0,131.0
...,...,...,...,...,...,...,...,...,...
6919,197,eunpyeong,unknown,2020-09-01,restaurant,37.608417,126.933753,101.0,99.0
6920,124,dongdaemoon,Dongdaemoon,2020-09-01,supermarket,37.562248,127.067210,153.0,198.0
6921,189,eunpyeong,overseas inflow,2020-09-01,convenience_store,37.615341,126.915245,93.0,85.0
6922,124,dongdaemoon,Dongdaemoon,2020-09-01,convenience_store,37.566120,127.066683,149.0,198.0


In [27]:
# 저장
final_path = join(root, 'data', 'extracted', 'merged_route_final.csv')
final_df.to_csv(final_path, encoding='utf-8-sig', index=False)